In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os

# Show the current working directory
current_directory = os.getcwd()
current_directory

'/content'

In [ ]:
import pandas as pd

# Load the two CSV files
file1 = "./data/parsed_sentiments.csv"
file2 = "./data/parsed_sentiments2.csv"

# Read the files into pandas DataFrames
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# Merge the DataFrames
merged_df = pd.concat([df1, df2], ignore_index=True)

# Save the merged DataFrame to a new CSV file
output_file = "./data/merged_sentiments.csv"
merged_df.to_csv(output_file, index=False)

In [ ]:
# Drop rows where sentiment is not 'positive', 'negative', or 'neutral', considering case sensitivity
valid_sentiments = ['positive', 'negative', 'neutral']

# Standardize case of the 'sentiment' column and filter rows
filtered_df = merged_df[merged_df['sentiment'].str.lower().isin(valid_sentiments)]
filtered_df.loc[filtered_df['word'] == '绿色', 'sentiment'] = 'positive'
filtered_df.loc[filtered_df['word'] == '碳中和', 'sentiment'] = 'positive'
filtered_df.loc[filtered_df['word'] == '中共中央', 'sentiment'] = 'neutral'
filtered_df.loc[filtered_df['word'] == '社会主义', 'sentiment'] = 'negative'
filtered_df.loc[filtered_df['word'] == '生态', 'sentiment'] = 'positive'
filtered_df.loc[filtered_df['word'] == '低碳', 'sentiment'] = 'positive'
filtered_df.loc[filtered_df['word'] == '保护', 'sentiment'] = 'positive'
filtered_df.loc[filtered_df['word'] == '科学', 'sentiment'] = 'positive'
filtered_df.loc[filtered_df['word'] == '综合治理', 'sentiment'] = 'positive'
filtered_df.loc[filtered_df['word'] == '碳达峰', 'sentiment'] = 'positive'
filtered_df.loc[filtered_df['word'] == '电动', 'sentiment'] = 'positive'
filtered_df.loc[filtered_df['word'] == '。', 'sentiment'] = 'neutral'
filtered_df.loc[filtered_df['word'] == '、', 'sentiment'] = 'neutral'
filtered_df.loc[filtered_df['word'] == '，', 'sentiment'] = 'neutral'
filtered_df.loc[filtered_df['word'] == '贯彻', 'sentiment'] = 'neutral'


# Save the filtered DataFrame to a new CSV file
filtered_file = "./data/filtered_sentiments.csv"
filtered_df.to_csv(filtered_file, index=False)

<ipython-input-4-c5d8bb197a95>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[filtered_df['word'] == '绿色', 'sentiment'] = 'positive'
<ipython-input-4-c5d8bb197a95>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[filtered_df['word'] == '碳中和', 'sentiment'] = 'positive'
<ipython-input-4-c5d8bb197a95>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.loc[filtered_df['word'] == '中共中央', 'sentimen

In [5]:
pip install transformers datasets torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [26]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup

In [ ]:
import pandas as pd

# Load the dataset
file_path = "./data/filtered_sentiments.csv"
df = pd.read_csv(file_path)

df['sentiment'] = df['sentiment'].str.lower()
# Map sentiment labels to numerical values
label_mapping = {'positive': 2, 'neutral': 1, 'negative': 0}
df['label'] = df['sentiment'].map(label_mapping)
# Define the labels want to keep
desired_labels = ['positive', 'neutral', 'negative']

# Filter the DataFrame
df = df[df['sentiment'].isin(desired_labels)].reset_index(drop=True)

# Save the processed dataset to ensure consistency
processed_file = "./data/processed_sentiments.csv"
df[['word', 'label']].rename(columns={'word': 'text'}).to_csv(processed_file, index=False)


In [59]:
class SentimentDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_len):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=self.max_len,
            truncation=True,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


In [ ]:
df = pd.read_csv('./data/processed_sentiments.csv')

# Preview the data
print(df.head())

   text  label
0  中共中央      1
1   国务院      1
2    关于      1
3    加快      2
4  经济社会      0


In [61]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'], df['label'], test_size=0.1, random_state=42
)


In [62]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
MAX_LEN = 128


In [63]:
train_dataset = SentimentDataset(train_texts.tolist(), train_labels.tolist(), tokenizer, MAX_LEN)
val_dataset = SentimentDataset(val_texts.tolist(), val_labels.tolist(), tokenizer, MAX_LEN)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16)



In [64]:
model = BertForSequenceClassification.from_pretrained('bert-base-chinese', num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [65]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [66]:
batch_size = 16
EPOCHS = 3
learning_rate = 2e-5

In [67]:
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)

total_steps = len(train_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [68]:
from torch.nn.functional import cross_entropy

def train_epoch(model, data_loader, optimizer, device, scheduler):
    model.train()
    total_loss = 0

    for batch in data_loader:
        optimizer.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        scheduler.step()

    avg_loss = total_loss / len(data_loader)
    return avg_loss


In [69]:
def eval_model(model, data_loader, device):
    model.eval()
    total_acc = 0

    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, preds = torch.max(outputs.logits, dim=1)
            total_acc += torch.sum(preds == labels)

    avg_acc = total_acc.double() / len(data_loader.dataset)
    return avg_acc


In [70]:

for epoch in range(EPOCHS):
    print(f'Epoch {epoch + 1}/{EPOCHS}')
    train_loss = train_epoch(model, train_loader, optimizer, device, scheduler)
    val_acc = eval_model(model, val_loader, device)

    print(f'Train loss: {train_loss}')
    print(f'Validation accuracy: {val_acc}')


Epoch 1/3
Train loss: 1.0466750366459585
Validation accuracy: 0.415041782729805
Epoch 2/3
Train loss: 0.9691426728178959
Validation accuracy: 0.5506035283194057
Epoch 3/3
Train loss: 0.7826654389746511
Validation accuracy: 0.6035283194057567


In [ ]:
#model.save_pretrained('/content/drive/MyDrive/Lab1/fine-tuned-bert-environmental-sentiment')
#tokenizer.save_pretrained('/content/drive/MyDrive/Lab1/fine-tuned-bert-environmental-sentiment')

('/content/drive/MyDrive/Lab1/fine-tuned-bert-environmental-sentiment/tokenizer_config.json',
 '/content/drive/MyDrive/Lab1/fine-tuned-bert-environmental-sentiment/special_tokens_map.json',
 '/content/drive/MyDrive/Lab1/fine-tuned-bert-environmental-sentiment/vocab.txt',
 '/content/drive/MyDrive/Lab1/fine-tuned-bert-environmental-sentiment/added_tokens.json')

In [53]:
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform labels
df['label'] = label_encoder.fit_transform(df['label'])

# Verify label encoding
print("Encoded labels:", df['label'].unique())

Encoded labels: [1 2 0]


In [73]:
import torch.nn.functional as F

def predict_weighted_score(text):
    encoding = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=MAX_LEN,
        truncation=True,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt',
    )

    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    outputs = model(input_ids, attention_mask)
    logits = outputs.logits

    # Apply softmax to get probabilities
    probabilities = F.softmax(logits, dim=1)

    # Compute weighted sum
    class_indices = torch.arange(3).to(device)  # [0, 1, 2]
    score = torch.sum(probabilities * class_indices, dim=1).item()
    return score

# Example usage
new_text = "你的环境保护工作做得很好。"
print(predict_weighted_score(new_text))


1.7969516515731812


In [54]:
df['label'] = df['sentiment'].map({'negative': 0, 'neutral': 1, 'positive': 2})
print(encoded_dataset['train'])


Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 21533
})


In [56]:
df['sentiment'].unique()

array(['neutral', 'positive', 'negative'], dtype=object)